# ***NOTEBOOK EN GBQ: "Cargas de trabajo - etl_vl"***

# INICIALIZACIÓN

In [ ]:
# @title INSTALACIÓN DE LIBRERÍAS
!pip install boto3
!pip install google-cloud-secret-manager


In [ ]:
# @title IMPORTACIÓN DE LIBRERÍAS GCP

import boto3
from google.cloud import storage
import os

from google.cloud import bigquery  # Importa el cliente de BigQuery
from google.cloud import storage   # Importa el cliente de Cloud Storage
import pandas as pd
import io
import re
import unicodedata
import chardet  # Biblioteca para detectar codificaciones de texto

from google.colab import auth
auth.authenticate_user()

from google.cloud import secretmanager
import os

# Configura el cliente de Secret Manager
client = secretmanager.SecretManagerServiceClient()
project_id = "animum-dev-datawarehouse"

# Función para obtener un secreto por su nombre
def get_secret(secret_id):
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

# Obtén los valores de los secretos
hs_datawarehouse_secret_token = get_secret("hs_datawarehouse_secret_token")

# Configura las variables de entorno
os.environ['hs_datawarehouse_secret_token'] = hs_datawarehouse_secret_token

print("Secretos configurados correctamente.")

Secretos configurados correctamente.


In [ ]:
# @title S3_folder_and_files_list()
def S3_folder_and_files_list(params: dict) -> dict:
    """
    Lista todos los archivos y subcarpetas a partir de una carpeta específica en un bucket de s3.

    Args:
        params (dict): Diccionario con las claves:
            - S3_bucket_name (str): Nombre del bucket de s3.
            - S3_folder_path (str): Ruta de la carpeta en el bucket de s3.

    Returns:
        dict: Diccionario con la estructura de carpetas y archivos:
            {
                'folders': {
                    'subcarpeta1/': {
                        'files': [lista de archivos en subcarpeta1],
                        'folders': {estructura recursiva de subcarpetas}
                    }
                },
                'files': [lista de archivos en la carpeta actual]
            }

    Raises:
        ValueError: Si falta algún parámetro en params.
        Exception: Si ocurre un error al listar los objetos.
    """
    import boto3
    import logging

    # Configurar el registro
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Validar parámetros
    S3_bucket_name = params.get('S3_bucket_name')
    S3_folder_path = params.get('S3_folder_path')

    if not S3_bucket_name or not S3_folder_path:
        raise ValueError("Faltan parámetros requeridos: 'S3_bucket_name' o 'S3_folder_path'.")

    try:
        # Cliente de s3
        s3_client = boto3.client('s3')

        paginator = s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=S3_bucket_name, Prefix=S3_folder_path)

        structure = {'folders': {}, 'files': []}

        # Procesar los objetos obtenidos
        for page in pages:
            for obj in page.get('Contents', []):
                key = obj['Key']
                if key.endswith('/'):
                    # Es una carpeta
                    relative_path = key[len(S3_folder_path):]
                    if '/' not in relative_path.strip('/'):
                        structure['folders'][key] = {
                            'files': [],
                            'folders': {}
                        }
                else:
                    # Es un archivo
                    relative_path = key[len(S3_folder_path):]
                    if '/' not in relative_path:
                        structure['files'].append(key)

        return structure

    except Exception as e:
        logger.error(f"Error al listar objetos en s3: {e}")
        raise

In [ ]:
# @title S3_to_GCS_transfer_file()
def S3_to_GCS_transfer_file(config: dict):
    """
    Transfiere archivos desde un bucket S3 a un bucket de Google Cloud Storage.
    Muestra mensajes por pantalla en tiempo real.

    Args:
        config (dict): Diccionario con las claves:
            - S3_bucket_name (str): Nombre del bucket en S3.
            - S3_folder_path (str): Ruta de la carpeta en el bucket de S3.
            - GCS_bucket_name (str): Nombre del bucket en Google Cloud Storage.
            - filter_file (dict): Diccionario con opciones de filtro para los archivos:
                - use_bool (bool): Activar o desactivar los filtros.
                - name_include_patterns_list (list[str]): Lista de patrones a incluir por nombre.
                - name_exclude_patterns_list (list[str]): Lista de patrones a excluir por nombre.
                - extension_include_patterns_list (list[str]): Extensiones permitidas.
                - extension_exclude_patterns_list (list[str]): Extensiones excluidas.
                - min_size_kb (int): Tamaño mínimo del archivo en KB.
                - max_size_kb (int): Tamaño máximo del archivo en KB.
                - modified_after_date (str): Fecha mínima de modificación (YYYY-MM-DD).
                - modified_before_date (str): Fecha máxima de modificación (YYYY-MM-DD).
                - include_subfolders_bool (bool): Incluir subcarpetas en el filtro.
    """
    import boto3
    from google.cloud import storage
    import os
    from datetime import datetime

    start_time = datetime.now()
    print(f"\n=== Iniciando proceso de transferencia de archivos | {start_time} ===\n", flush=True)

    # Extraer parámetros
    S3_bucket_name = config['S3_bucket_name']
    S3_folder_path = config['S3_folder_path']
    GCS_bucket_name = config['GCS_bucket_name']
    filter_file = config.get('filter_file', {})

    print("Autenticando cliente de s3...", flush=True)
    s3_client = boto3.client('s3')

    print("Autenticando cliente de Google Cloud Storage...", flush=True)
    storage_client = storage.Client()
    bucket = storage_client.bucket(GCS_bucket_name)

    # Listar objetos en la carpeta s3
    print(f"Listando archivos en s3 bucket: '{S3_bucket_name}', carpeta: '{S3_folder_path}'\n", flush=True)
    s3_objects = s3_client.list_objects_v2(Bucket=S3_bucket_name, Prefix=S3_folder_path)

    if 'Contents' not in s3_objects:
        print("No se encontraron objetos en la carpeta s3 especificada.\n", flush=True)
        return

    # Extraer la lista de archivos (claves)
    s3_files = []
    for obj in s3_objects['Contents']:
        key = obj['Key']
        # Subcarpetas
        if not filter_file.get('include_subfolders_bool', True):
            relative_path = key[len(S3_folder_path):]
            if '/' in relative_path:
                continue
        s3_files.append(key)

    # Aplicar filtros si están habilitados
    if filter_file.get('use_bool', False):
        print("Aplicando filtros definidos en 'filter_file':", flush=True)
        print(filter_file, flush=True)

        # Filtrar por patrones en el nombre
        include_patterns = filter_file.get('name_include_patterns_list', [])
        exclude_patterns = filter_file.get('name_exclude_patterns_list', [])

        if include_patterns:
            before_count = len(s3_files)
            s3_files = [
                file for file in s3_files
                if any(pattern in os.path.basename(file) for pattern in include_patterns)
            ]
            after_count = len(s3_files)
            print(f"Filtrando por include_patterns={include_patterns}: {before_count} -> {after_count}", flush=True)

        if exclude_patterns:
            before_count = len(s3_files)
            s3_files = [
                file for file in s3_files
                if not any(pattern in os.path.basename(file) for pattern in exclude_patterns)
            ]
            after_count = len(s3_files)
            print(f"Filtrando por exclude_patterns={exclude_patterns}: {before_count} -> {after_count}", flush=True)

        # Filtrar por extensión
        include_extensions = filter_file.get('extension_include_patterns_list', [])
        exclude_extensions = filter_file.get('extension_exclude_patterns_list', [])

        if include_extensions:
            before_count = len(s3_files)
            s3_files = [file for file in s3_files if any(file.endswith(ext) for ext in include_extensions)]
            after_count = len(s3_files)
            print(f"Filtrando extensiones permitidas={include_extensions}: {before_count} -> {after_count}", flush=True)

        if exclude_extensions:
            before_count = len(s3_files)
            s3_files = [file for file in s3_files if not any(file.endswith(ext) for ext in exclude_extensions)]
            after_count = len(s3_files)
            print(f"Excluyendo extensiones={exclude_extensions}: {before_count} -> {after_count}", flush=True)

        # Filtrar por tamaño
        min_size_kb = filter_file.get('min_size_kb')
        max_size_kb = filter_file.get('max_size_kb')
        if min_size_kb or max_size_kb:
            print(f"Filtrando por tamaño: min_size_kb={min_size_kb}, max_size_kb={max_size_kb}", flush=True)
            before_count = len(s3_files)
            all_objects = {obj['Key']: obj for obj in s3_objects['Contents']}
            filtered_files = []
            for file in s3_files:
                obj = all_objects.get(file)
                if not obj:
                    continue
                file_size_kb = obj['Size'] / 1024
                if (min_size_kb and file_size_kb < min_size_kb):
                    continue
                if (max_size_kb and file_size_kb > max_size_kb):
                    continue
                filtered_files.append(file)
            s3_files = filtered_files
            after_count = len(s3_files)
            print(f"Filtrando por tamaño: {before_count} -> {after_count}", flush=True)

        # Filtrar por fecha de modificación
        modified_after = filter_file.get('modified_after_date')
        modified_before = filter_file.get('modified_before_date')
        if modified_after or modified_before:
            print(f"Filtrando por fecha. after={modified_after}, before={modified_before}", flush=True)
            before_count = len(s3_files)
            from datetime import datetime
            modified_after_dt = datetime.strptime(modified_after, "%Y-%m-%d") if modified_after else None
            modified_before_dt = datetime.strptime(modified_before, "%Y-%m-%d") if modified_before else None

            all_objects = {obj['Key']: obj for obj in s3_objects['Contents']}
            filtered_files = []
            for file in s3_files:
                obj = all_objects.get(file)
                if not obj:
                    continue
                last_modified = obj['LastModified']

                if modified_after_dt and last_modified < modified_after_dt:
                    continue
                if modified_before_dt and last_modified > modified_before_dt:
                    continue
                filtered_files.append(file)

            s3_files = filtered_files
            after_count = len(s3_files)
            print(f"Filtrado por fecha: {before_count} -> {after_count}", flush=True)

    # Verificar si hay archivos por transferir
    if not s3_files:
        print("No se encontraron archivos para transferir tras aplicar filtros.\n", flush=True)
        end_time = datetime.now()
        print(f"=== Proceso finalizado sin transferencias | {end_time} ===\n", flush=True)
        return

    print("=== Comenzando a transferir archivos ===\n", flush=True)
    transfer_log = []

    for s3_file in s3_files:
        temp_file_name = os.path.basename(s3_file)
        temp_file_path = f"/tmp/{temp_file_name}"

        print(f"Iniciando transferencia de '{s3_file}'", flush=True)
        try:
            # Descargar archivo desde s3
            print(f"Descargando: s3://{S3_bucket_name}/{s3_file} -> {temp_file_path}", flush=True)
            s3_client.download_file(S3_bucket_name, s3_file, temp_file_path)
            print(f"Archivo descargado correctamente: {temp_file_path}", flush=True)

            # Subir archivo a GCS
            GCS_file_name = temp_file_name
            print(f"Subiendo: {temp_file_path} -> gs://{GCS_bucket_name}/{GCS_file_name}", flush=True)
            blob = bucket.blob(GCS_file_name)
            blob.upload_from_filename(temp_file_path)
            print(f"Archivo subido correctamente: gs://{GCS_bucket_name}/{GCS_file_name}", flush=True)

            transfer_log.append({
                'archivo_s3': f"s3://{S3_bucket_name}/{s3_file}",
                'archivo_GCS': f"gs://{GCS_bucket_name}/{GCS_file_name}",
                'estado': 'Transferido con éxito'
            })
        except Exception as e:
            print(f"Error al transferir '{s3_file}': {e}", flush=True)
            transfer_log.append({
                'archivo_s3': f"s3://{S3_bucket_name}/{s3_file}",
                'archivo_GCS': None,
                'estado': f"Error: {e}"
            })
        finally:
            # Eliminar archivo temporal
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)
                print(f"Eliminado archivo temporal: {temp_file_path}\n", flush=True)

    # Resumen final
    end_time = datetime.now()
    print(f"=== Proceso de transferencia finalizado | {end_time} ===", flush=True)
    print(f"Duración total: {end_time - start_time}", flush=True)
    print("\n=== Informe de transferencia ===", flush=True)
    for log in transfer_log:
        print(f"- {log['archivo_s3']} -> {log['archivo_GCS']} | Estado: {log['estado']}", flush=True)


In [ ]:
# @title GCS_load_CSV_to_BQ_dic
import io
import re
import pandas as pd
import unicodedata
from google.cloud import storage, bigquery
import chardet
import datetime
import os

def GCS_load_CSV_to_BQ_dic(params: dict) -> dict:
    """
    Carga uno o varios archivos CSV desde un bucket de GCS a una o varias tablas en BigQuery.
    Filtra los archivos según los parámetros especificados y realiza la conversión a UTF-8 si es necesario.
    Además, guarda los nombres originales como descripciones en el esquema de BigQuery.

    Args:
        params (dict): Configuración de carga:
            - project_id (str): ID del proyecto en Google Cloud.
            - source_GCS_bucket_name (str): Nombre del bucket de GCS.
            - source_GCS_file_names_list (list[str]): Lista de nombres de archivos CSV. Si está vacío, se importan todos.
            - source_GCS_file_names_filter (dict): Opciones de filtro para los archivos:
                - use_bool (bool): Activar o desactivar los filtros.
                - name_include_patterns_list (list[str]): Patrones a incluir por nombre.
                - name_exclude_patterns_list (list[str]): Patrones a excluir por nombre.
                - extension_include_patterns_list (list[str]): Extensiones permitidas.
                - extension_exclude_patterns_list (list[str]): Extensiones excluidas.
                - min_size_kb (int): Tamaño mínimo del archivo en KB.
                - max_size_kb (int): Tamaño máximo del archivo en KB.
                - modified_after_date (str): Fecha mínima de modificación (YYYY-MM-DD).
                - modified_before_date (str): Fecha máxima de modificación (YYYY-MM-DD).
                - include_subfolders_bool (bool): Incluir subcarpetas.
            - destination_GBQ_dataset_id (str): ID del dataset de BigQuery.

    Returns:
        dict: Información del resultado para cada archivo, incluyendo:
            - Mensaje de éxito.
            - ID de la tabla de destino.
            - Número de filas cargadas.
            - Mapeo de nombres originales a normalizados.

    Raises:
        ValueError: Si faltan parámetros obligatorios o si un archivo no existe en el bucket.
    """

    def normalize_column_name(name: str) -> str:
        """Normaliza los nombres de columnas para que sean válidos en BigQuery."""
        name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
        name = name.replace('ñ', 'n').replace('Ñ', 'N')
        normalized = re.sub(r"[^a-zA-Z0-9_]", "_", name)
        normalized = re.sub(r"_+", "_", normalized).strip("_")
        return normalized[:300]

    def normalize_table_name(name: str) -> str:
        """Normaliza el nombre del archivo para ser un ID de tabla válido en BigQuery."""
        return normalize_column_name(name.replace(".", "_"))

    def file_passes_filters(blob) -> bool:
        """Determina si un archivo cumple los filtros especificados."""
        if not filter_file.get("use_bool", False):
            return True

        # Verificar patrones de nombre
        blob_name = blob.name
        if filter_file.get("name_include_patterns_list") and not any(pat in blob_name for pat in filter_file["name_include_patterns_list"]):
            return False
        if filter_file.get("name_exclude_patterns_list") and any(pat in blob_name for pat in filter_file["name_exclude_patterns_list"]):
            return False

        # Verificar extensiones
        extension = os.path.splitext(blob_name)[1]
        if filter_file.get("extension_include_patterns_list") and extension not in filter_file["extension_include_patterns_list"]:
            return False
        if filter_file.get("extension_exclude_patterns_list") and extension in filter_file["extension_exclude_patterns_list"]:
            return False

        # Verificar tamaño
        size_kb = blob.size / 1024
        if filter_file.get("min_size_kb") and size_kb < filter_file["min_size_kb"]:
            return False
        if filter_file.get("max_size_kb") and size_kb > filter_file["max_size_kb"]:
            return False

        # Verificar fechas de modificación
        modified_date = blob.updated.date()
        if filter_file.get("modified_after_date"):
            after_date = datetime.datetime.strptime(filter_file["modified_after_date"], "%Y-%m-%d").date()
            if modified_date < after_date:
                return False
        if filter_file.get("modified_before_date"):
            before_date = datetime.datetime.strptime(filter_file["modified_before_date"], "%Y-%m-%d").date()
            if modified_date > before_date:
                return False

        return True

    # Extraer y validar los parámetros
    project_id = params.get("project_id")
    bucket_name = params.get("source_GCS_bucket_name")
    file_names = params.get("source_GCS_file_names_list", [])
    dataset_id = params.get("destination_GBQ_dataset_id")
    filter_file = params.get("source_GCS_file_names_filter", {})

    if not all([project_id, bucket_name, dataset_id]):
        raise ValueError("❌ Faltan parámetros obligatorios: project_id, source_GCS_bucket_name, destination_GBQ_dataset_id.")

    storage_client = storage.Client(project=project_id)
    bq_client = bigquery.Client(project=project_id)
    results = {}

    # Listar blobs y convertir a lista para evitar problemas de iteración
    bucket = storage_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix="", delimiter="/" if not filter_file.get("include_subfolders_bool", False) else None))

    # Si no se proporcionaron nombres específicos, considerar todos los archivos del bucket
    if not file_names:
        file_names = [blob.name for blob in blobs]

    filtered_files = [blob.name for blob in blobs if file_passes_filters(blob) and blob.name in file_names]

    for file_name in filtered_files:
        print(f"\n🔄 Procesando archivo: {file_name}")

        # Verificar la existencia del archivo en el bucket
        blob = bucket.blob(file_name)
        if not blob.exists():
            raise ValueError(f"❌ El archivo '{file_name}' no existe en el bucket '{bucket_name}'.")

        # Detectar codificación
        raw_data = blob.download_as_bytes()
        encoding = chardet.detect(raw_data)['encoding']
        print(f"🔍 Codificación detectada: {encoding}")

        # Leer el archivo CSV
        try:
            raw_text = raw_data.decode(encoding, errors="replace")
            print("✔️ Archivo leído correctamente.")
            df = pd.read_csv(io.StringIO(raw_text), delimiter=";")
        except Exception as e:
            print(f"❌ Error al procesar el archivo '{file_name}': {e}")
            continue  # Saltar al siguiente archivo

        # Convertir a UTF-8
        utf8_file_path = f"/tmp/{file_name.replace('.csv', '_utf8.csv')}"
        try:
            df.to_csv(utf8_file_path, index=False, sep=";", encoding="utf-8")
            print(f"📁 Archivo convertido a UTF-8: {utf8_file_path}")
        except Exception as e:
            print(f"❌ Error al convertir el archivo a UTF-8: {e}")
            continue  # Saltar al siguiente archivo

        # Normalizar columnas y generar mapeo
        original_columns = df.columns.tolist()
        normalized_columns = [normalize_column_name(col) for col in original_columns]
        column_mapping = dict(zip(original_columns, normalized_columns))

        print("⚠️ Mapeo de nombres de columnas:")
        for original, normalized in column_mapping.items():
            print(f"   - `{original}` → `{normalized}`")

        # Actualizar los nombres de las columnas
        df.columns = normalized_columns
        df.to_csv(utf8_file_path, index=False, sep=";")

        # Crear esquema de BigQuery
        schema = [
            bigquery.SchemaField(col, "STRING", description=f"Nombre original: {original}")
            for original, col in column_mapping.items()
        ]

        # Normalizar nombre de la tabla
        table_name = normalize_table_name(file_name.split('.')[0])
        table_ref = bq_client.dataset(dataset_id).table(table_name)
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            skip_leading_rows=1,
            field_delimiter=";",
            quote_character='"',
            allow_quoted_newlines=True,
            encoding="UTF-8",
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
            schema=schema,
            max_bad_records=10
        )

        print(f"📤 Cargando datos a la tabla `{dataset_id}.{table_name}`...")
        try:
            with open(utf8_file_path, "rb") as file_obj:
                load_job = bq_client.load_table_from_file(file_obj, table_ref, job_config=job_config)
                result = load_job.result()

            # Obtener detalles de la tabla
            table = bq_client.get_table(table_ref)

            print("\n=== Estadísticas finales de la tabla ===")
            print(f"✅ Total de filas cargadas: {result.output_rows}")
            print(f"✅ Total de columnas: {len(table.schema)}")

            if load_job.errors:
                print(f"⚠️ Filas ignoradas debido a errores: {len(load_job.errors)}")
                for error in load_job.errors:
                    print(f"   - {error['message']}")
            else:
                print("✅ Sin errores durante la carga.")

            results[file_name] = {
                "message": "Carga completada exitosamente.",
                "table_id": table.table_id,
                "output_rows": result.output_rows,
                "column_mapping": column_mapping
            }

        except Exception as e:
            print(f"❌ Error durante la carga del archivo '{file_name}': {e}")
            continue  # Saltar al siguiente archivo

    return results



# EJECUCIONES

In [ ]:
# @title LISTADO DE ARCHIVOS S3
S3_bucket_name = "animum-datalake-landing-euwest3-637423635409" # @param {"type":"string"}
S3_folder_path = "velneo/" # @param {"type":"string"}
params = {
    'S3_bucket_name': S3_bucket_name,
    'S3_folder_path': S3_folder_path
}

# Ejecutar la función
result = S3_folder_and_files_list(params)

import pprint

# Mostrar resultado
pprint.pprint(result)

{'files': ['velneo/ESC-ANOS-utf8.csv',
           'velneo/ESC-ANOS.csv',
           'velneo/ESC-APET-utf8.csv',
           'velneo/ESC-APET.csv',
           'velneo/ESC-AULA-utf8.csv',
           'velneo/ESC-AULA.csv',
           'velneo/ESC-C&M-utf8.csv',
           'velneo/ESC-C&M.csv',
           'velneo/ESC-C&P-utf8.csv',
           'velneo/ESC-C&P.csv',
           'velneo/ESC-CALE-utf8.csv',
           'velneo/ESC-CALE.csv',
           'velneo/ESC-COBR-utf8.csv',
           'velneo/ESC-COBR.csv',
           'velneo/ESC-CONT-utf8.csv',
           'velneo/ESC-CONT.csv',
           'velneo/ESC-CONV-utf8.csv',
           'velneo/ESC-CONV.csv',
           'velneo/ESC-CURS-utf8.csv',
           'velneo/ESC-CURS.csv',
           'velneo/ESC-DIAS-utf8.csv',
           'velneo/ESC-DIAS.csv',
           'velneo/ESC-DISC-utf8.csv',
           'velneo/ESC-DISC.csv',
           'velneo/ESC-EVAC-utf8.csv',
           'velneo/ESC-EVAC.csv',
           'velneo/ESC-EVAL-utf8.csv',
           'veln

In [ ]:
# @title COPIADO DE ARCHIVOS S3 A GCS

S3_bucket_name = "animum-datalake-landing-euwest3-637423635409"  # @param {"type":"string"}
S3_folder_path = "velneo/"  # @param {"type":"string"}
GCS_bucket_name = "erp_velneo"  # @param {"type":"string"}

use_bool = True  # @param {"type": "boolean"}
name_include_patterns_list = ["utf8"]  # @param {"type": "raw"}
name_exclude_patterns_list = ["backup"]  # @param {"type": "raw"}
extension_include_patterns_list = [".txt", ".csv"]  # @param {"type": "raw"}
extension_exclude_patterns_list = [".log"]  # @param {"type": "raw"}
min_size_kb = None  # @param {"type": "number"}
max_size_kb = None  # @param {"type": "number"}
modified_before_date = ""  # @param {"type": "date"}
modified_after_date = ""  # @param {"type":"date"}
include_subfolders_bool = True  # @param {"type": "boolean"}

filter_file = {
    "use_bool": use_bool,
    "name_include_patterns_list": name_include_patterns_list,
    "name_exclude_patterns_list": name_exclude_patterns_list,
    "extension_include_patterns_list": extension_include_patterns_list,
    "extension_exclude_patterns_list": extension_exclude_patterns_list,
    "min_size_kb": min_size_kb,
    "max_size_kb": max_size_kb,
    "modified_after_date": modified_after_date,
    "modified_before_date": modified_before_date,
    "include_subfolders_bool": include_subfolders_bool,
}

config = {
    "S3_bucket_name": S3_bucket_name,
    "S3_folder_path": S3_folder_path,
    "GCS_bucket_name": GCS_bucket_name,
    "filter_file": filter_file,
}

# Ejecutar
S3_to_GCS_transfer_file(config)


=== Iniciando proceso de transferencia de archivos | 2025-01-27 10:39:10.434874 ===

Autenticando cliente de s3...
Autenticando cliente de Google Cloud Storage...
Listando archivos en s3 bucket: 'animum-datalake-landing-euwest3-637423635409', carpeta: 'velneo/'

Aplicando filtros definidos en 'filter_file':
{'use_bool': True, 'name_include_patterns_list': ['utf8'], 'name_exclude_patterns_list': ['backup'], 'extension_include_patterns_list': ['.txt', '.csv'], 'extension_exclude_patterns_list': ['.log'], 'min_size_kb': None, 'max_size_kb': None, 'modified_after_date': '', 'modified_before_date': '', 'include_subfolders_bool': True}
Filtrando por include_patterns=['utf8']: 103 -> 51
Filtrando por exclude_patterns=['backup']: 51 -> 51
Filtrando extensiones permitidas=['.txt', '.csv']: 51 -> 51
Excluyendo extensiones=['.log']: 51 -> 51
=== Comenzando a transferir archivos ===

Iniciando transferencia de 'velneo/ESC-ANOS-utf8.csv'
Descargando: s3://animum-datalake-landing-euwest3-6374236354

In [ ]:
# @title CARGA DE GCS CSV A GBQ

# Configuración de parámetros
dic_params = {
    "project_id": "animum-dev-datawarehouse",
    "source_GCS_bucket_name": "erp_velneo",
    "source_GCS_file_names_list": ["ESC-C&M-utf8.csv", "ESC-C&P-utf8.csv", "ESC-L&T-utf8.csv"],  # Si está vacío, procesará todos los archivos
    "source_GCS_file_names_filter": {
        "use_bool": False,
        "name_include_patterns_list": ["ENTIDADES"],
        "extension_include_patterns_list": [".csv"],
        "min_size_kb": 10,
        "max_size_kb": 5000,
        "modified_after_date": "2023-01-01",
        "modified_before_date": "2025-12-31",
        "include_subfolders_bool": False
    },
    "destination_GBQ_dataset_id": "vl_01raw_01"
}

# Ejecutar con validaciones
try:
    result = GCS_load_CSV_to_BQ_dic(dic_params)  # Actualizado el nombre de la función
    print("\n=== Resultado de la carga ===")
    for file_name, details in result.items():
        print(f"\nArchivo: {file_name}")
        print(f"   - Mensaje: {details['message']}")
        print(f"   - Tabla destino: {details['table_id']}")
        print(f"   - Filas cargadas: {details['output_rows']}")
        print(f"   - Mapeo de columnas: {details['column_mapping']}")
except Exception as e:
    print(f"Error durante la carga: {e}")



🔄 Procesando archivo: ESC-C&M-utf8.csv
🔍 Codificación detectada: utf-8
✔️ Archivo leído correctamente.
📁 Archivo convertido a UTF-8: /tmp/ESC-C&M-utf8_utf8.csv
⚠️ Mapeo de nombres de columnas:
   - `Curso` → `Curso`
   - `Módulo` → `Modulo`
   - `Orden` → `Orden`
   - `Siguiente Módulo` → `Siguiente_Modulo`
   - `Recibos por Módulo` → `Recibos_por_Modulo`
   - `Fecha creación` → `Fecha_creacion`
   - `Hora creación` → `Hora_creacion`
   - `Usuario creación` → `Usuario_creacion`
   - `Fecha modificación` → `Fecha_modificacion`
   - `Hora modificación` → `Hora_modificacion`
   - `Usuario modificación` → `Usuario_modificacion`
📤 Cargando datos a la tabla `vl_01raw_01.ESC_C_M_utf8`...

=== Estadísticas finales de la tabla ===
✅ Total de filas cargadas: 1734
✅ Total de columnas: 11
✅ Sin errores durante la carga.

🔄 Procesando archivo: ESC-C&P-utf8.csv
🔍 Codificación detectada: utf-8
✔️ Archivo leído correctamente.
📁 Archivo convertido a UTF-8: /tmp/ESC-C&P-utf8_utf8.csv
⚠️ Mapeo de nombre